In [17]:
import pandas as pd 
import numpy as np
import requests
import time
import re
from pathlib import Path
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import os



## Problemas de desigualdad social

In [26]:
# URL de la API de la pagina https://www.economia.gob.mx/datamexico/es/vizbuilder 
url = ("https://www.economia.gob.mx/apidatamexico/tesseract/data.jsonrecords"
       "?Year=2010%2C2012%2C2014%2C2016%2C2018%2C2020%2C2022"
       "&cube=coneval_gini_ent"
       "&drilldowns=Year%2CState"
       "&locale=es"
       "&measures=GINI")

resp = requests.get(url)
resp.raise_for_status() 
data = resp.json()

# Lo convertimos en un data frame
df = pd.DataFrame(data["data"])

#  colocar los nombres de las columnas en español
df = df.rename(columns={
    "Year": "anio",
    "State ID": "id estado",
    "State": "estado",
    "GINI": "gini"
})
# mas cerca del 1 mayor desigualdad social

# datos numericos 
df["anio"] = pd.to_numeric(df["anio"], errors="coerce")
df["gini"] = pd.to_numeric(df["gini"], errors="coerce")

# Ordenar
df = df.sort_values(["estado", "anio"]).reset_index(drop=True)

df.head(10)


,anio,id estado,estado,gini
0,2010,1,Aguascalientes,0.507408
1,2012,1,Aguascalientes,0.479191
2,2014,1,Aguascalientes,0.486294
3,2016,1,Aguascalientes,0.409074
4,2018,1,Aguascalientes,0.420223
5,2020,1,Aguascalientes,0.395063
6,2022,1,Aguascalientes,0.429214
7,2010,2,Baja California,0.505827
8,2012,2,Baja California,0.464538
9,2014,2,Baja California,0.433568


In [19]:
# año unico 
df["anio"].unique()

array([2010, 2012, 2014, 2016, 2018, 2020, 2022])

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   anio       224 non-null    int64  
 1   id estado  224 non-null    int64  
 2   estado     224 non-null    object 
 3   gini       224 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 7.1+ KB


### entre mas cerca del 1 se encuentre mayor desigualdad social, datos de https://www.economia.gob.mx/datamexico/es/vizbuilder
- 0 (Cero): Representa la igualdad perfecta. Esto significaría que todas las personas en una sociedad tienen exactamente los mismos ingresos. -
- 1 (Uno): Representa la desigualdad perfecta. Esto ocurriría si una sola persona tuviera todos los ingresos y los demás no tuvieran nada.

## Estimado del 2019 al 2024 de rezago por nivel educativo datos del https://www.gob.mx/inea/documentos/rezago-educativo 

In [ ]:
# # el io es para manejar archivos en memoria. y el time para poner pausas entre descargas
# from io import BytesIO
# import time
# # las url son de los archivos excel en la pagina https://www.gob.mx/inea/documentos/rezago-educativo
# archivos = {
#     2024: "https://www.gob.mx/cms/uploads/attachment/file/976883/est_rez_2024_actualizado.xlsx",
#     2023: "https://www.gob.mx/cms/uploads/attachment/file/976882/est_rez_2023_actualizado.xlsx",
#     2022: "https://www.gob.mx/cms/uploads/attachment/file/976881/est_rez_2022_actualizado.xlsx",
#     2021: "https://www.gob.mx/cms/uploads/attachment/file/976880/est_rez_2021_actualizado.xlsx",
#     2020: "https://www.gob.mx/cms/uploads/attachment/file/976878/est_rez_2020_actualizado.xlsx",
#     2019: "https://www.gob.mx/cms/uploads/attachment/file/976879/est_rez_2019_actualizado.xlsx"
# }


# dataframes = []
# # descargamos cada archivo, limpiamos y agregamos una columna con el año
# for year, url in archivos.items():
#     print(f"Descargando {year}: {url}")
#     response = requests.get(url)
#     if response.status_code == 200:
#         df_raw = pd.read_excel(BytesIO(response.content), header=None)
#         df_clean = df_raw.drop(index=range(0, 5)).reset_index(drop=True)
#         df_clean["Año"] = year
#         dataframes.append(df_clean)
#     else:
#         print(f"Error al descargar {year}: {response.status_code}")
#     time.sleep(2)

# if dataframes:
#     df_total = pd.concat(dataframes, ignore_index=True)
#     df_total.to_excel("rezago_educativo_2019_2024.xlsx", index=False)
#     print("Archivos combinados guardados en 'rezago_educativo_2019_2024.xlsx'")
# else:
#     print("No se descargaron archivos.")



In [ ]:


# # en casoo de que no se hayan descargado los archivos no se descarguen con el web scrapping
# archivos = {
#     2024: "https://www.gob.mx/cms/uploads/attachment/file/976883/est_rez_2024_actualizado.xlsx",
#     2023: "https://www.gob.mx/cms/uploads/attachment/file/976882/est_rez_2023_actualizado.xlsx",
#     2022: "https://www.gob.mx/cms/uploads/attachment/file/976881/est_rez_2022_actualizado.xlsx",
#     2021: "https://www.gob.mx/cms/uploads/attachment/file/976880/est_rez_2021_actualizado.xlsx",
#     2020: "https://www.gob.mx/cms/uploads/attachment/file/976878/est_rez_2020_actualizado.xlsx",
#     2019: "https://www.gob.mx/cms/uploads/attachment/file/976879/est_rez_2019_actualizado.xlsx"
# }

# # Carpeta donde guardarás los archivos descargados
# carpeta = "archivos_rezago"
# os.makedirs(carpeta, exist_ok=True)

# dataframes = []

# for year, url in archivos.items():
#     nombre_archivo = f"{carpeta}/est_rez_{year}.xlsx"
    
#     # Verificar si ya existe el archivo
#     if os.path.exists(nombre_archivo):
#         print(f" {year} ya existe. Cargando desde disco...")
#         df_raw = pd.read_excel(nombre_archivo, header=None)
#     else:
#         print(f"⬇ Descargando {year} desde {url}")
#         response = requests.get(url)
#         if response.status_code == 200:
#             with open(nombre_archivo, "wb") as f:
#                 f.write(response.content)
#             df_raw = pd.read_excel(BytesIO(response.content), header=None)
#         else:
#             print(f"Error al descargar {year}: {response.status_code}")
#             continue
    
#     df_clean = df_raw.drop(index=range(0, 5)).reset_index(drop=True)
#     df_clean["Año"] = year
#     dataframes.append(df_clean)
    
#     time.sleep(2)

# if dataframes:
#     df_total = pd.concat(dataframes, ignore_index=True)
#     df_total.to_excel("rezago_educativo_2019_2024.xlsx", index=False)
#     print(" Archivos combinados guardados en 'rezago_educativo_2019_2024.xlsx'")
# else:
#     print("No se descargaron archivos.")


In [ ]:
# df_total.head(10)

## Debemos limpiar los archivos ya que el docuemnto en excel tiene filas que solo contienen titulos o texto por lo que si la entidad federativa es cero  elimina las columans 

In [ ]:

# nombre_archivo_entrada = "rezago_educativo_2019_2024.xlsx"
# nombre_archivo_salida = "rezago_educativo_LIMPIO.csv" 
# try:
#     # Cargar el archivo de Excel, indicando que no tiene una fila de encabezado.
#     df = pd.read_excel(nombre_archivo_entrada, header=None)


#     columnas_correctas = [
#         'Num', 'Entidad', 'Poblacion_15_y_mas',
#         'Analfabetas_Abs', 'Analfabetas_Lugar', 'Analfabetas_Porc', 'Analfabetas_Porc_Lugar',
#         'Sin_Primaria_Abs', 'Sin_Primaria_Lugar', 'Sin_Primaria_Porc', 'Sin_Primaria_Porc_Lugar',
#         'Sin_Secundaria_Abs', 'Sin_Secundaria_Lugar', 'Sin_Secundaria_Porc', 'Sin_Secundaria_Porc_Lugar',
#         'Rezago_Total_Abs', 'Rezago_Total_Lugar', 'Rezago_Total_Porc', 'Rezago_Total_Porc_Lugar',
#         'Año'
#     ]
#     df.columns = columnas_correctas
#     valores_a_eliminar = ['Entidad', 'Estados Unidos Mexicanos']
#     df_limpio = df[~df['Entidad'].isin(valores_a_eliminar)]

#     # Eliminar cualquier fila que no tenga un nombre del estado
#     df_limpio = df_limpio.dropna(subset=['Entidad'])

#     # reiniciar el índice del DataFrame.
#     df_limpio = df_limpio.reset_index(drop=True)
    
#     # comvertir las columnas de datos a tipo numérico.
#     columnas_numericas = [
#         'Poblacion_15_y_mas', 'Analfabetas_Abs', 'Analfabetas_Porc',
#         'Sin_Primaria_Abs', 'Sin_Primaria_Porc', 'Sin_Secundaria_Abs',
#         'Sin_Secundaria_Porc', 'Rezago_Total_Abs', 'Rezago_Total_Porc'
#     ]
#     # Convertir las columnas a tipo numérico,
#     for col in columnas_numericas:
#         df_limpio[col] = pd.to_numeric(df_limpio[col], errors='coerce')

#  # Guardar el DataFrame limpio en un nuevo archivo CSV
#     df_limpio.to_csv(nombre_archivo_salida, index=False)
# except FileNotFoundError:
#     print(f"ERROR: No se encontró el archivo '{nombre_archivo_entrada}'.")

In [ ]:
# df_limpio.head()

In [ ]:
# df_limpio.info()

## Lo mismo pero con web scrapping

In [ ]:
# # from selenium import webdriver
# # from selenium.webdriver.common.by import By
# # from selenium.webdriver.chrome.options import Options
# # import os
# # import time

# # Ruta actual del script
# download_dir = Path(r'C:\Users\joel_\seguridadDesarrollo\seguridadDesarrollo\data\raw')
# # Configura opciones de descarga para Chrome
# options = Options()

# options.add_experimental_option("prefs", {
#     "download.default_directory": str(download_dir),
#     "download.prompt_for_download": False,
#     "download.directory_upgrade": True,
#     "safebrowsing.enabled": True
# })

# # abre la ventanita del navegador
# driver = webdriver.Chrome(options=options)

# # Abre la página
# driver.get("https://www.gob.mx/inea/documentos/rezago-educativo")
# time.sleep(3)  # Espera a que cargue la página

# # archivos xsls los guarda en la variables botones 
# botones = driver.find_elements(By.XPATH, "//a[contains(@onclick, 'Estimación del rezago educativo al 31 de diciembre de')]")

# # Hace clic en cada enlace que comience con "Estimación del rezago educativo al 31 de diciembre de"
# for boton in botones:
#     onclick = boton.get_attribute("onclick")
#     if onclick:
#         print(f"Descargando archivo con onclick: {onclick}")
#         driver.execute_script("arguments[0].click();", boton)
#         time.sleep(2)  # Espera para que dé tiempo a descargar

# print(f"Todos los archivos se han descargado en: {download_dir}")
# #cierra la ventanita del navegador
# driver.quit() 



In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import os
import time


#  los datos se deben guardar en la carpeta raw ya que son datos sin procesar
download_dir = Path(r'C:\Users\joel_\seguridadDesarrollo\seguridadDesarrollo\data\raw')

# Crea el directorio si no existe.
download_dir.mkdir(parents=True, exist_ok=True)

print(f"Se configurará la descarga en la carpeta: {download_dir}")

# Configura las opciones de descarga para Chrome.
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": str(download_dir),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# Abre el navegador y la página web.
driver = webdriver.Chrome(options=options)
driver.get("https://www.gob.mx/inea/documentos/rezago-educativo")
time.sleep(3)

# 5. Encuentra y hace clic en cada enlace de descarga.
botones = driver.find_elements(By.XPATH, "//a[contains(@onclick, 'Estimación del rezago educativo al 31 de diciembre de')]")

for boton in botones:
    onclick = boton.get_attribute("onclick")
    if onclick:
        print(f"Iniciando descarga para: {onclick.split(',')[1]}")
        driver.execute_script("arguments[0].click();", boton)
        time.sleep(2)

print("\nTodas las descargas han sido iniciadas.")

# 6. Cierra el navegador.
driver.quit()

print("Esperando a que los archivos finalicen la descarga...")
time.sleep(5)

print("\nIniciando el procesamiento de los archivos Excel...")

excel_files = list(download_dir.glob('*.xlsx'))

if not excel_files:
    print("\nNo se encontraron archivos de Excel para procesar.")
else:
    all_data = []
    for file_path in excel_files:
        print(f"Procesando el archivo: {file_path.name}")
        match = re.search(r'(\d{4})', file_path.name)
        if match:
            year = int(match.group(1))

            #  las primeras 8 filas no son datos útiles ya que contiene titulos y otras cosas que no son datos
            df = pd.read_excel(file_path, skiprows=8)
 
            
            df['Año'] = year
            all_data.append(df)

    # Concatena y limpia los datos.
    merged_df = pd.concat(all_data, ignore_index=True)
    
    merged_df.dropna(subset=['Entidad federativa'], inplace=True)
    merged_df = merged_df[~merged_df['Entidad federativa'].str.contains('Total', na=False)]

    # Guarda el resultado final.
    output_path = download_dir / 'rezago_educativo_consolidado.csv'
    merged_df.to_csv(output_path, index=False)
    print(f"El archivo final se guardó en: '{output_path}'")

Se configurará la descarga en la carpeta: C:\Users\joel_\seguridadDesarrollo\seguridadDesarrollo\data\raw
Iniciando descarga para:  'Estimación del rezago educativo al 31 de diciembre de 2024 por entidad federativa')
Iniciando descarga para:  'Estimación del rezago educativo al 31 de diciembre de 2023 por entidad federativa')
Iniciando descarga para:  'Estimación del rezago educativo al 31 de diciembre de 2022 por entidad federativa')
Iniciando descarga para:  'Estimación del rezago educativo al 31 de diciembre de 2021 por entidad federativa')
Iniciando descarga para:  'Estimación del rezago educativo al 31 de diciembre de 2020 por entidad federativa')
Iniciando descarga para:  'Estimación del rezago educativo al 31 de diciembre de 2019 por entidad federativa')

Todas las descargas han sido iniciadas.
Esperando a que los archivos finalicen la descarga...

Iniciando el procesamiento de los archivos Excel...
Procesando el archivo: est_rez_2019_actualizado.xlsx
Procesando el archivo: est_

In [22]:
data_dir = Path(r'C:\Users\joel_\seguridadDesarrollo\seguridadDesarrollo\data\raw')
file_path = data_dir / 'rezago_educativo_consolidado.csv'
df_consolidado = pd.read_csv(file_path)
df_consolidado.head()




,Entidad federativa,Unnamed: 1,Población de 15 años y más,Analfabetas,Lugar,%,Lugar.1,Sin primaria terminada,Lugar.2,%.1,Lugar.3,Sin secundaria terminada,Lugar.4,%.2,Lugar.5,Rezago total,Lugar.6,%.3,Lugar.7,Año
0,1,Aguascalientes,1041764.0,22065.0,30.0,2.118042,27.0,69670.0,28.0,6.687695,26.0,157318.0,28.0,15.101117,22.0,249053.0,29.0,23.906854,26.0,2019
1,2,Baja California,2896404.0,52779.0,21.0,1.822225,29.0,199512.0,15.0,6.888266,24.0,464047.0,13.0,16.021487,13.0,716338.0,13.0,24.731978,24.0,2019
2,3,Baja California Sur,602639.0,14047.0,32.0,2.330915,26.0,42528.0,32.0,7.056961,23.0,89170.0,31.0,14.796586,24.0,145745.0,32.0,24.184462,25.0,2019
3,4,Campeche,690317.0,40543.0,26.0,5.873099,9.0,66569.0,29.0,9.643251,13.0,99723.0,30.0,14.445972,28.0,206835.0,30.0,29.962322,11.0,2019
4,5,Coahuila de Zaragoza,2326265.0,39349.0,27.0,1.691510,30.0,127339.0,22.0,5.473968,30.0,350332.0,17.0,15.059849,23.0,517020.0,19.0,22.225327,30.0,2019


In [23]:
df_consolidado.head(10)

,Entidad federativa,Unnamed: 1,Población de 15 años y más,Analfabetas,Lugar,%,Lugar.1,Sin primaria terminada,Lugar.2,%.1,Lugar.3,Sin secundaria terminada,Lugar.4,%.2,Lugar.5,Rezago total,Lugar.6,%.3,Lugar.7,Año
0,1,Aguascalientes,1041764.0,22065.0,30.0,2.118042,27.0,69670.0,28.0,6.687695,26.0,157318.0,28.0,15.101117,22.0,249053.0,29.0,23.906854,26.0,2019
1,2,Baja California,2896404.0,52779.0,21.0,1.822225,29.0,199512.0,15.0,6.888266,24.0,464047.0,13.0,16.021487,13.0,716338.0,13.0,24.731978,24.0,2019
2,3,Baja California Sur,602639.0,14047.0,32.0,2.330915,26.0,42528.0,32.0,7.056961,23.0,89170.0,31.0,14.796586,24.0,145745.0,32.0,24.184462,25.0,2019
3,4,Campeche,690317.0,40543.0,26.0,5.873099,9.0,66569.0,29.0,9.643251,13.0,99723.0,30.0,14.445972,28.0,206835.0,30.0,29.962322,11.0,2019
4,5,Coahuila de Zaragoza,2326265.0,39349.0,27.0,1.691510,30.0,127339.0,22.0,5.473968,30.0,350332.0,17.0,15.059849,23.0,517020.0,19.0,22.225327,30.0,2019
5,6,Colima,560968.0,18952.0,31.0,3.378446,18.0,49572.0,31.0,8.836868,16.0,88709.0,32.0,15.813558,14.0,157233.0,31.0,28.028872,18.0,2019
6,7,Chiapas,3784323.0,520490.0,2.0,13.753847,1.0,514623.0,4.0,13.598813,2.0,786274.0,7.0,20.777138,2.0,1821387.0,4.0,48.129798,1.0,2019
7,8,Chihuahua,2810358.0,73849.0,16.0,2.627744,24.0,209270.0,14.0,7.446382,21.0,499209.0,11.0,17.763182,9.0,782328.0,12.0,27.837308,19.0,2019
8,9,Ciudad de México,7605589.0,108642.0,11.0,1.428450,32.0,335640.0,9.0,4.413070,32.0,927538.0,5.0,12.195479,32.0,1371820.0,8.0,18.036999,32.0,2019
9,10,Durango,1323834.0,36463.0,28.0,2.754348,23.0,112898.0,24.0,8.528109,17.0,221834.0,24.0,16.756935,12.0,371195.0,25.0,28.039392,17.0,2019


In [24]:
df_consolidado.drop(columns=['Entidad federativa'], inplace=True)
df_consolidado.rename(columns={'Unnamed: 1': 'entidad_federativa'}, inplace=True)
df_consolidado

,entidad_federativa,Población de 15 años y más,Analfabetas,Lugar,%,Lugar.1,Sin primaria terminada,Lugar.2,%.1,Lugar.3,Sin secundaria terminada,Lugar.4,%.2,Lugar.5,Rezago total,Lugar.6,%.3,Lugar.7,Año
0,Aguascalientes,1041764.0,22065.0,30.0,2.118042,27.0,69670.0,28.0,6.687695,26.0,157318.0,28.0,15.101117,22.0,249053.0,29.0,23.906854,26.0,2019
1,Baja California,2896404.0,52779.0,21.0,1.822225,29.0,199512.0,15.0,6.888266,24.0,464047.0,13.0,16.021487,13.0,716338.0,13.0,24.731978,24.0,2019
2,Baja California Sur,602639.0,14047.0,32.0,2.330915,26.0,42528.0,32.0,7.056961,23.0,89170.0,31.0,14.796586,24.0,145745.0,32.0,24.184462,25.0,2019
3,Campeche,690317.0,40543.0,26.0,5.873099,9.0,66569.0,29.0,9.643251,13.0,99723.0,30.0,14.445972,28.0,206835.0,30.0,29.962322,11.0,2019
4,Coahuila de Zaragoza,2326265.0,39349.0,27.0,1.691510,30.0,127339.0,22.0,5.473968,30.0,350332.0,17.0,15.059849,23.0,517020.0,19.0,22.225327,30.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Veracruz de Ignacio de la Llave,6291017.0,476591.0,1.0,7.575739,4.0,751796.0,2.0,11.950309,3.0,1121695.0,3.0,17.830106,6.0,2350082.0,2.0,37.356154,5.0,2024
200,Yucatán,1938136.0,103258.0,11.0,5.327696,8.0,174494.0,16.0,9.003187,7.0,272711.0,20.0,14.070788,20.0,550463.0,18.0,28.401670,9.0,2024
201,Zacatecas,1239086.0,38837.0,23.0,3.134326,15.0,108186.0,22.0,8.731113,8.0,211759.0,25.0,17.089936,7.0,358782.0,24.0,28.955375,8.0,2024
202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024


In [28]:
df_consolidado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   entidad_federativa          192 non-null    object 
 1   Población de 15 años y más  192 non-null    float64
 2   Analfabetas                 192 non-null    float64
 3   Lugar                       192 non-null    float64
 4   %                           192 non-null    float64
 5   Lugar.1                     192 non-null    float64
 6   Sin primaria terminada      192 non-null    float64
 7   Lugar.2                     192 non-null    float64
 8   %.1                         192 non-null    float64
 9   Lugar.3                     192 non-null    float64
 10  Sin secundaria terminada    192 non-null    float64
 11  Lugar.4                     192 non-null    float64
 12  %.2                         192 non-null    float64
 13  Lugar.5                     192 non